# Project 1 — E‑Commerce EDA & A/B Stats

In [ ]:
import pandas as pd
import numpy as np
customers = pd.read_csv("../data_raw/customers.csv", parse_dates=["signup_date"])
products = pd.read_csv("../data_raw/products.csv")
orders = pd.read_csv("../data_raw/orders.csv", parse_dates=["order_date"])
order_items = pd.read_csv("../data_raw/order_items.csv")
sessions = pd.read_csv("../data_raw/sessions.csv", parse_dates=["session_date"])
customers.head()

## KPI: AOV zilnic

In [ ]:
aov_daily = (orders.assign(day=orders["order_date"].dt.date)
             .groupby("day")
             .agg(orders=("order_id","nunique"), revenue=("revenue","sum"), aov=("revenue","mean"))
             .reset_index())
aov_daily.head()

## Cohorts

In [ ]:
orders["order_month"] = orders["order_date"].dt.to_period("M")
first_order = (orders.groupby("customer_id")["order_date"].min().dt.to_period("M").rename("cohort"))
orders = orders.join(first_order, on="customer_id")
orders["cohort_index"] = (orders["order_month"] - orders["cohort"]).apply(lambda x: x.n)
cohort_pivot = (orders.groupby(["cohort","cohort_index"])["customer_id"].nunique().unstack(fill_value=0))
cohort_pivot.head()

## A/B Test — Conversie

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
tab = pd.crosstab(sessions["is_variant_B"], sessions["converted"])
chi2, p, dof, exp = chi2_contingency(tab)
p